# Práctica 4: Procesamiento del Lenguaje Natural

__Fecha de entrega: 14 de mayo de 2024__

El objetivo de esta práctica es aplicar los conceptos teóricos vistos en clase en el módulo de PLN.

Lo más importante en esta práctica no es el código Python, sino el análisis de los datos y modelos que construyas y las explicaciones razonadas de cada una de las decisiones que tomes. __No se valorarán trozos de código o gráficas sin ningún tipo de contexto o explicación__.

Finalmente, recuerda establecer el parámetro `random_state` en todas las funciones que tomen decisiones aleatorias para que los resultados sean reproducibles (los resultados no varíen entre ejecuciones).

In [ ]:
# Ziteng Huang y Lubin Ye

RANDOM_STATE = 1234

In [ ]:
import nltk
import re
import numpy as np
import csv
import pandas as pd

from sklearn.model_selection import train_test_split
from collections import Counter

# 1) Carga del conjunto de datos

El fichero `spam.csv` contiene mensajes SMS etiquetados como spam or ham (legítimo).

Muestra un ejemplo de cada clase.

Haz un estudio del conjunto de datos. ¿qué palabras aparecen más veces?, ¿tendría sentido normalizar de alguna manera el corpus?

Crea una partición de los datos dejando el 60% para entrenamiento, 20% para validación y el 20% restante para test. Comprueba que la distribución de los ejemplos en las particiones es similar. 

RESPUESTA:



In [ ]:
# Cargar dataset -> encoding="latin1"

def load_data():
    corpus = []
    labels = []
    with open('spam.csv', 'r', encoding="latin1") as f:
        reader = csv.reader(f)
        reader.__next__()
        for row in reader:
            corpus.append(row[1])
            labels.append(row[0])

    corpus_df = pd.DataFrame({'Document': corpus, 
                        'Category': labels})

    return corpus, labels, corpus_df


corpus, labels, corpus_df = load_data()

corpus_df


In [ ]:
# Contar palabras más frecuentes -> palabras sin significado

def contar_palabras(corpus):
    lista_palabras = []
    for text in corpus:
        lista_palabras.extend(text.split(" "))

    counter = Counter(lista_palabras)
    palabras_mas_comunes = counter.most_common()

    # Palabras más comunes
    print("Palabras más comunes:")
    for palabra, frecuencia in palabras_mas_comunes:
        print(f"{palabra}: {frecuencia}")

contar_palabras(corpus)

In [ ]:
wpt = nltk.WordPunctTokenizer()
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)


In [ ]:
# Borrar palabras sin signifcado (stop words)

norm_corpus = normalize_corpus(corpus)
norm_corpus

In [ ]:
# Volvemos a contar ahora las palabras más comunes después de normalizar

contar_palabras(norm_corpus)

In [ ]:
# Split data en training, validacion y test

def split_data(corpus, labels):
    
    corpus_train, corpus_val_test, labels_train, labels_val_test = train_test_split(corpus, labels, test_size=0.4, random_state=RANDOM_STATE)
    corpus_val, corpus_test, labels_val, labels_test = train_test_split(corpus_val_test, labels_val_test, test_size=0.5, random_state=RANDOM_STATE)

    corpus_df_train = pd.DataFrame({'Document': corpus_train, 
                           'Category': labels_train})
    
    corpus_df_val = pd.DataFrame({'Document': corpus_val,
                                    'Category': labels_val})
                                    
    corpus_df_test = pd.DataFrame({'Document': corpus_test,
                                    'Category': labels_test})

    return corpus_df_train, corpus_df_val, corpus_df_test


corpus_df_train, corpus_df_val, corpus_df_test = split_data(norm_corpus, labels)
corpus_df_train

In [ ]:
import matplotlib.pyplot as plt

def plot_class_distribution(corpus_df, title):
    plt.figure(figsize=(6,6))
    corpus_df['Category'].value_counts().plot(kind='bar', color=['skyblue', 'orange'])
    plt.title(title)
    plt.xlabel('Category')
    plt.ylabel('Count')
    plt.show()

plot_class_distribution(corpus_df_train, "Training dataset")
plot_class_distribution(corpus_df_val, "Validation dataset")
plot_class_distribution(corpus_df_test, "Test dataset")


# 2) Representación como bolsa de palabras

Elige justificadamente una representación de bolsa de palabras y aplícala.
Muestra un ejemplo antes y después de aplicar la representación. Explica los cambios.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def get_tfidf_features(corpus):
    tv = TfidfVectorizer()
    tv_matrix = tv.fit_transform(corpus)
    tv_matrix = tv_matrix.toarray()

    vocab = tv.get_feature_names_out()
    return pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)

# Ejemplo de frecuencia de las palabras de los primeros 5 documentos
print(get_tfidf_features(norm_corpus[:5]))

# Ejemplo de frecuencia de todos los documentos
print(get_tfidf_features(norm_corpus))

# 3) Aplica 3 algoritmos de aprendizaje automático para resolver la tarea

Justifica porqué los has elegido.
Ajusta los modelos respecto a un hiperparámetro que consideres oportuno. Justifica tu elección.
Explica los resultados obtenidos.

In [ ]:
# Árbol de decisión 

from sklearn import tree
import numpy as np

vectorizer = TfidfVectorizer()
train_preprocessed = vectorizer.fit_transform(corpus_df_train["Document"]).toarray()
test_preprocessed = vectorizer.transform(corpus_df_test["Document"]).toarray()

# Creamos el clasificador con los valores por defecto
tree_classifier = tree.DecisionTreeClassifier()
tree_classifier.fit(train_preprocessed, corpus_df_train["Category"])

tree_train_predictions = tree_classifier.predict(train_preprocessed)
tree_test_predictions = tree_classifier.predict(test_preprocessed)

print("Árbol, porcentaje de aciertos en entrenamiento:", np.mean(tree_train_predictions == corpus_df_train["Category"]))
print("Árbol, porcentaje de aciertos en test:", np.mean(tree_test_predictions == corpus_df_test["Category"]))

In [ ]:
# k Vecinos Más Cercanos (k-NN)

from sklearn import neighbors

knn_classifier = neighbors.KNeighborsClassifier()
knn_classifier.fit(train_preprocessed, corpus_df_train["Category"])

knn_train_predictions = knn_classifier.predict(train_preprocessed)
knn_test_predictions = knn_classifier.predict(test_preprocessed)

print("k-NN, porcentaje de aciertos en entrenamiento:", np.mean(knn_train_predictions == corpus_df_train["Category"]))
print("k-NN, porcentaje de aciertos en test:", np.mean(knn_test_predictions == corpus_df_test["Category"]))

In [ ]:
# Naive Bayes

from sklearn.naive_bayes import MultinomialNB

mnb_classifier = MultinomialNB()

mnb_classifier.fit(train_preprocessed, corpus_df_train["Category"])

mnb_train_predictions = mnb_classifier.predict(train_preprocessed)
mnb_test_predictions = mnb_classifier.predict(test_preprocessed)

print("Multinomial Naive Bayes, porcentaje de aciertos en entrenamiento:", np.mean(mnb_train_predictions == corpus_df_train["Category"]))
print("Multinomial Naive Bayes, porcentaje de aciertos en test:", np.mean(mnb_test_predictions == corpus_df_test["Category"]))

# 4) Construye redes neuronales con Keras con distintas maneras de usar word embeddings

Justifica tus decisiones y explica los resultados obtenidos.

# 5) Aplica los modelos construidos a los datos de test y compáralos.

Calcula las métricas de recall, precisión y f1.
Discute cual es el mejor modelo y cual es peor y porqué.